In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! pip install -q transformers datasets sentencepiece

!pip uninstall -y -q transformers accelerate
!pip install -q transformers accelerate

!pip install -q rouge-score

!pip install -q torchmetrics

!pip install -q git+https://github.com/huggingface/accelerate

!git clone -q https://github.com/huggingface/transformers.git
!cd transformers
!pip install  -q -e.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 30.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: file:///content does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [3]:

model_name_or_path = 'google/mt5-small'
# model_name_or_path = '/content/drive/MyDrive/mt5-pquad/checkpoint-11764'
from transformers import AutoTokenizer, MT5ForConditionalGeneration
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = MT5ForConditionalGeneration.from_pretrained(model_name_or_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [12]:
from datasets import concatenate_datasets, load_dataset

train_data = load_dataset("Gholamreza/pquad", split="train")
valid_data = load_dataset("Gholamreza/pquad", split="validation")
test_data = load_dataset("Gholamreza/pquad", split="test")

concat_dataset = concatenate_datasets([train_data, valid_data, test_data])
dataset=concat_dataset

In [13]:
dataset

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 79972
})

In [14]:
del concat_dataset, train_data, valid_data

small_dataset = dataset

small_dataset= dataset.filter(lambda example: len(example['answers']['text'])>0)
# small_dataset= small_dataset.filter(lambda example: len(example['context'])<600)
small_dataset= small_dataset.filter(lambda example: 'چرا' not in example['context'])
split_data = small_dataset.train_test_split(test_size=0.1, seed= 42, shuffle=True)

dataset = split_data

del split_data

Filter:   0%|          | 0/79972 [00:00<?, ? examples/s]

Filter:   0%|          | 0/60356 [00:00<?, ? examples/s]

In [15]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 52936
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5882
    })
})

In [16]:
# def format_dataset(example):
#   return {'input': 'question: ' + example['question'] + ' context: ' + example['context'], 'target': example['answers'][0]['text']}

def format_dataset(example):
  context = example['context']
  question = example['question']
  answer = example['answers']['text'][0]
  question_plus = f"question: {str(question)}"
  question_plus += f" context: {str(context)} </s>"

  answer_plus = answer
  answer_plus = f"{answer_plus} </s>"

  # return {'input': 'question: ' + example['question'] + ' context: ' + example['context'], 'target': example['answers'][0]['text']}
  return {'input': question_plus, 'target': answer_plus}

dataset = dataset.map(format_dataset, remove_columns=dataset['train'].column_names)

Map:   0%|          | 0/52936 [00:00<?, ? examples/s]

Map:   0%|          | 0/5882 [00:00<?, ? examples/s]

In [17]:
test_data = dataset['test']

test_data[0]

# train_ds=  dataset['train'].map(encode)
# valid_ds=  dataset['test'].map(encode)


{'input': 'question: تاریخ کتابت مجموعهٔ مواعظ و مجالس مولانا چه سالی می\u200cباشد؟ context: مجموعهٔ مواعظ و مجالس مولانا یعنی سخنانی است که به وجه اندرز و به طریق تذکیر بر سر منبر بیان کرده\u200cاست. نسخهٔ خطی این کتاب در کتابخانهٔ سلیم آقا دراسگدار محفوظ و در تاریخ کتابت آن سال ۷۸۸ می\u200cباشد. در این خطبه\u200cها رومی آیات قرآن و احادیث را تفسیر می\u200cکند. وی همچنین از اشعار سنائی، عطار و دیگر شاعران، من جمله آثار خود در این خطبه\u200cها استفاده می\u200cکند. همان\u200cطور که افلاکی روایت می\u200cکند، پس از شمس تبریزی، رومی به درخواست بزرگان، به ویژه صلاح الدین زرکوب، خطبه\u200cها را ایراد می\u200cکرده\u200cاست. این خطابه\u200cها به فارسی ساده است، اما نقل قول\u200cهای عربی و روایت او از تاریخ و احادیث، نشان از تبحر وی در علوم اسلامی دارد. </s>',
 'target': '۷۸۸ </s>'}

In [18]:
def convert_to_features(example_batch):
    input_encodings = tokenizer.batch_encode_plus(example_batch['input'], truncation=True, padding='max_length', max_length=564)
    target_encodings = tokenizer.batch_encode_plus(example_batch['target'], truncation=True, padding='max_length', max_length=64)

    encodings = {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids'],
        'decoder_attention_mask': target_encodings['attention_mask']
    }

    return encodings

In [19]:
dataset = dataset.map(convert_to_features, batched=True, remove_columns=dataset['train'].column_names)

columns = ['input_ids', 'attention_mask', 'labels', 'decoder_attention_mask']

# train_ds.set_format(type='torch', columns=columns)
# valid_ds.set_format(type='torch', columns=columns)

dataset.set_format(type='torch', columns=columns)

Map:   0%|          | 0/52936 [00:00<?, ? examples/s]

Map:   0%|          | 0/5882 [00:00<?, ? examples/s]

In [20]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq,Seq2SeqTrainingArguments, Seq2SeqTrainer
# data_collator = default_data_collator
data_collator = DataCollatorForSeq2Seq(tokenizer)

In [21]:
from transformers import Seq2SeqTrainer
from transformers import Seq2SeqTrainingArguments

# set training arguments - Feel free to adapt it
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/mT5_QA",
    per_device_train_batch_size=4,
    num_train_epochs=1,
    per_device_eval_batch_size=4,
    predict_with_generate=True,
    evaluation_strategy="epoch",
    do_train=True,
    do_eval=True,
    save_strategy="epoch"
    # save_strategy="no",
    #eval_steps=1000,
    # push_to_hub=False
    #fp16=True,
)


In [22]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    # data_collator=T2TDataCollator(),
    data_collator=data_collator,
    tokenizer=tokenizer,
    # compute_metrics=compute_metrics
)

In [23]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [24]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,0.143800,0.078145


TrainOutput(global_step=13234, training_loss=1.2212483574837127, metrics={'train_runtime': 4897.9111, 'train_samples_per_second': 10.808, 'train_steps_per_second': 2.702, 'total_flos': 3.083261720887296e+16, 'train_loss': 1.2212483574837127, 'epoch': 1.0})

In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# prompt = "question: خداوند در طرف راست کدام کوه با بنی اسرائیل وعده گذارد؟ context: اي بني اسرائيل ما شما را از (چنگال) دشمنتان نجات داديم، و در طرف راست كوه طور با شما وعده گذارديم، و من و سلوي بر شما نازل كرديم.د"

context = "ي بني اسرائيل ما شما را از (چنگال) دشمنتان نجات داديم، و در طرف راست كوه طور با شما وعده گذارديم، و من و سلوي بر شما نازل كرديم.د"

question = "خداوند در طرف راست کدام کوه با بنی اسرائیل وعده گذارد؟"

# context = """We went on a trip to Europe. We had our breakfast at 7 am in the morning at \
# the nearby coffee shop. Wore a dark blue over coat for our first visit to Louvre Museum \
# to experience history and art."""

# question = "At what time did we had breakfast?"

prompt = f"question: {question} context: {context} </s>"

input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

generated_ids = model.generate(input_ids,
                                    num_beams=7,
                                    num_return_sequences=7,
                                    top_k=50,
#                                     top_p=.2,
                                    temperature=0.95,

                                    max_length=200)
for generated in generated_ids:
    generated_text = tokenizer.decode(generated,skip_special_tokens=True)
print(generated_text)



/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


كوه طور با شما وعده گذارديم، و من و سلوي بر شما نازل کرديم 


In [26]:
context = """ي بني اسرائيل ما شما را از (چنگال) دشمنتان نجات داديم،  \
و در طرف راست كوه طور با شما وعده گذارديم، و من و سلوي بر شما نازل كرديم.د"""

question = "خداوند در طرف راست کدام کوه با بنی اسرائیل وعده گذارد؟"
print(context)
print(question)


# context = """We went on a trip to Europe. We had our breakfast at 7 am in the morning at \
# the nearby coffee shop. Wore a dark blue over coat for our first visit to Louvre Museum \
# to experience history and art."""

# question = "At what time did we had breakfast?"
# print(context)
# print(question)

ي بني اسرائيل ما شما را از (چنگال) دشمنتان نجات داديم،  و در طرف راست كوه طور با شما وعده گذارديم، و من و سلوي بر شما نازل كرديم.د
خداوند در طرف راست کدام کوه با بنی اسرائیل وعده گذارد؟


In [32]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

input_text =  f"answer: {question} context: {context} </s>"
input_ids = tokenizer(input_text,
                         return_tensors='pt', pad_to_max_length=True, truncation=True, max_length=64).input_ids.to(device)
# input_ids = encoded_query["input_ids"]
# attention_mask = input_ids["attention_mask"]
attention_mask = tokenizer(input_text,
                         return_tensors='pt', pad_to_max_length=True, truncation=True, max_length=64).attention_mask.to(device)

generated_answer = model.generate(input_ids, attention_mask = attention_mask,
                                 max_length=64, top_p=0.95, top_k=50)
for each in generated_answer:
  decoded_answer = tokenizer.decode(each, skip_special_tokens=True)
print("Answer: ", decoded_answer)

Answer:  پلاه طور با شما وعده گذارديم 
